In [7]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas_datareader as pdr
from datetime import datetime
import pandas as pd
import numpy as np
import streamlit as st

# just so we don't have to deal with scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
def getting_the_info():
    ticker = input('Enter A Valid Stock Ticker:  ')
    
    # going to have to set these in date time type
    start_date = input('Enter A Starting Date.\nFormat YYYY/MM/DD:  ')
    start_date = datetime.strptime(start_date,'%Y/%m/%d').date()
    end_date = input('Enter An Ending Date. \nFormat YYYY/MM/DD:  ')
    end_date = datetime.strptime(end_date,'%Y/%m/%d').date()
    
    period = int(input('Enter A Rolling Average Time Period In Days:  '))
    start_balance = int(input('Enter A Starting Balance Ammount:  '))
    years = (end_date - start_date).days / 365.25
    
    return ticker, start_date, end_date, period, start_balance, years

In [3]:
def calculating_metrics(stock, years):
    # metrics for the base
    base_return_metric = round((stock['Baseline'][-1] / stock['Baseline'][0] - 1) * 100,3)
    base_cagr_metric = round((((stock['Baseline'][-1] / stock['Baseline'][0]) ** (1/years)) - 1) * 100, 3)
    base_dd_metric = round(((stock['Base_DD'] / stock['Base_Peak']).min() * 100), 3)
    
    # metrics for the Moving Average Strat
    ma_return_metric = round((stock['SMA_Strat'][-1] / stock['SMA_Strat'][0] - 1) * 100,3)
    ma_cagr_metric = round((((stock['SMA_Strat'][-1] / stock['SMA_Strat'][0]) ** (1/years)) - 1) * 100, 3)
    ma_dd_metric = round(((stock['MA_DD'] / stock['MA_Peak']).min() * 100), 3)
    
    return base_return_metric, base_cagr_metric, base_dd_metric, ma_return_metric, ma_cagr_metric, ma_dd_metric

In [8]:
def sma_eval():
    ticker, start_date, end_date, period, start_balance, years = getting_the_info()
    
    # getting the df based on user stock ticker
    stock = pdr.get_data_yahoo(ticker, start_date, end_date)
    
    # dropping extra columns
    stock.drop(['High', 'Low', 'Volume', 'Adj Close'], axis = 1, inplace = True)
    
    # getting the daily return 
    stock['Return'] = stock['Close'] / stock['Close'].shift(1)
    
    # dropping the first row now because it has a NaN
    stock.dropna(inplace=True)
    
    # getting the baseline (if you didn't buy or sell at all after initial purchase)
    stock['Baseline'] = stock['Return'].cumprod() * start_balance
    
    # adding in the columns for baseline peak & drawdown
    stock['Base_Peak'] = stock['Baseline'].cummax()
    stock['Base_DD'] = stock['Baseline'] - stock['Base_Peak']
    
    # getting the Simple Moving Average
    stock['SMA'] = stock['Close'].rolling(window=period).mean()
    
    # making a column to let us know when to buy and when to sell
    stock['Buy'] = (stock['Close'] > stock['SMA']) * 1
    
    # Keep in mind we will only start buying after we get the first SMA so not until the period time we got before
    stock['Updated_Return'] = np.where(stock['Buy'].shift(1) == 1, stock['Return'], 1.0)
    stock.dropna(inplace = True) 
    
    # column to keep track of the moving average strategy
    stock['SMA_Strat'] = stock['Updated_Return'].cumprod() * start_balance
    
    # columns for the SMA metrics
    stock['MA_Peak'] = stock['SMA_Strat'].cummax()
    stock['MA_DD'] = stock['SMA_Strat'] - stock['MA_Peak']
    
    # getting the metrics from the function above now that all the columns are made
    base_return_metric, base_cagr_metric, base_dd_metric, ma_return_metric, ma_cagr_metric, ma_dd_metric = calculating_metrics(stock, years)
    
    # lets get rid of everything we've printed so far so it's not so messy looking
    clear_output()
    
  
    # Print out the metrics
    print('BASELINE')
    print(f'Return Metric: {base_return_metric}%')
    print('Compound Annual Growth Rate', base_cagr_metric)
    print(f'Biggest Drawdown was: {base_dd_metric}%')
    print(f'Total Return: {start_balance * (base_return_metric/100)}')
    print('---' * 15)
    print('MOVING AVERAGE STRATEGY')
    print(f'Return Metric: {ma_return_metric}%')
    print('Compound Annual Growth Rate', ma_cagr_metric)
    print(f'Biggest Drawdown was: {ma_dd_metric}%')
    print(f'Total Return: {start_balance * (ma_return_metric/100)}')
    
    # Print out the graph
    plt.figure(figsize=(12, 6))
    plt.plot(stock['Baseline'], label = 'Baseline')
    plt.plot(stock['SMA_Strat'], label = 'Moving Averge Strategy')
    plt.legend()
    return plt.show()

In [9]:
sma_eval()

,Open,Close,Return,Baseline,Base_Peak,Base_DD,SMA,Buy,Updated_Return,SMA_Strat,MA_Peak,MA_DD
Date,,,,,,,,,,,,
2011-04-13,5.02600,4.98600,1.01136,10435.32898,14847.21650,-4411.88752,4.74631,1,1.00000,10000.00000,10000.00000,0.00000
2011-04-14,4.97400,5.02800,1.00842,10523.23149,14847.21650,-4323.98501,4.74762,1,1.00842,10084.23549,10084.23549,0.00000
2011-04-15,5.13000,5.11600,1.01750,10707.40960,14847.21650,-4139.80690,4.75124,1,1.01750,10260.73027,10260.73027,0.00000
2011-04-18,5.02600,5.00600,0.97850,10477.18746,14847.21650,-4370.02904,4.75707,1,0.97850,10040.11228,10260.73027,-220.61799
2011-04-19,5.05200,5.03200,1.00519,10531.60359,14847.21650,-4315.61291,4.76612,1,1.00519,10092.25833,10260.73027,-168.47194
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,1073.67004,1093.93994,1.02525,2289535.30631,2574110.64379,-284575.33748,774.21760,1,1.02525,233098.01097,262070.67847,-28972.66749
2021-12-28,1109.48999,1088.46997,0.99500,2278087.06260,2574110.64379,-296023.58118,776.12025,1,0.99500,231932.46318,262070.67847,-30138.21529
2021-12-29,1098.64001,1086.18994,0.99791,2273315.13009,2574110.64379,-300795.51370,778.16680,1,0.99791,231446.63185,262070.67847,-30624.04662
